# 株価取得処理

In [1]:
# ライブラリのインストール
!pip install yfinance


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import yfinance as yf
import pandas as pd

# Pandasの表示オプションを設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# トヨタ自動車 (7203.T) のデータを取得
ticker_code = '7203.T'
ticker = yf.Ticker(ticker_code)

# 1. 株価データの取得 (過去1年分の日足)
print("--- 1. 株価データ (過去5日分) ---")
hist = ticker.history(period="1y")
print(hist.tail()) # .tail()で末尾5日分を表示

print("\n" + "="*50 + "\n")

# 2. 企業情報・指標の取得
print("--- 2. 企業情報・指標 ---")
info = ticker.info

# 必要な情報を抽出して表示
print(f"企業名: {info.get('longName')}")
print(f"業種: {info.get('industry')}")
print(f"株価: {info.get('currentPrice')}")
print(f"PER (株価収益率): {info.get('trailingPE')}")
print(f"PBR (株価純資産倍率): {info.get('priceToBook')}")
print(f"配当利回り: {info.get('dividendYield')}")

# 3. 配当性向の取得
# infoの中に'payoutRatio'というキーで配当性向が含まれている
print("\n--- 3. 配当性向 ---")
payout_ratio = info.get('payoutRatio')
if payout_ratio:
    print(f"配当性向: {payout_ratio:.2%}") # パーセント表示
else:
    print("配当性向データがありません。")

--- 1. 株価データ (過去5日分) ---
                             Open    High     Low   Close    Volume  Dividends  Stock Splits
Date                                                                                        
2025-06-09 00:00:00+09:00  2670.0  2672.0  2637.0  2640.0  13584200        0.0           0.0
2025-06-10 00:00:00+09:00  2653.0  2663.5  2640.0  2657.5  19556800        0.0           0.0
2025-06-11 00:00:00+09:00  2670.0  2678.0  2652.0  2656.0  16367500        0.0           0.0
2025-06-12 00:00:00+09:00  2645.0  2648.0  2608.0  2616.5  22784400        0.0           0.0
2025-06-13 00:00:00+09:00  2586.0  2589.0  2539.0  2555.0  36054800        0.0           0.0


--- 2. 企業情報・指標 ---
企業名: Toyota Motor Corporation
業種: Auto Manufacturers
株価: 2555.0
PER (株価収益率): 7.107291
PBR (株価純資産倍率): 0.9280495
配当利回り: 3.72

--- 3. 配当性向 ---
配当性向: 25.03%


In [2]:
import yfinance as yf
import pandas as pd

# Pandasの表示オプションを設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# トヨタ自動車 (7203.T) のデータを取得
ticker_code = '7203.T'
ticker = yf.Ticker(ticker_code)

# 1. 株価データの取得 (過去1年分の日足)
print("--- 1. 株価データ (過去5日分) ---")
# 1年分のデータを取得して、配当利回り計算に利用
hist = ticker.history(period="1y")
print(hist.tail()) # .tail()で末尾5日分を表示

print("\n" + "="*50 + "\n")

# 2. 企業情報・指標の取得
print("--- 2. 企業情報・指標 ---")
info = ticker.info

# 必要な情報を抽出して表示
print(f"企業名: {info.get('longName')}")
print(f"業種: {info.get('industry')}")
print(f"現在の株価: {info.get('currentPrice')}")
print(f"PER (株価収益率): {info.get('trailingPE'):.2f}")
print(f"PBR (株価純資産倍率): {info.get('priceToBook'):.2f}")

# 3. 配当利回りの1年単位での変動率
print("\n--- 3. 配当利回りの1年単位での変動率 ---")
# 過去の配当履歴を取得
dividends = ticker.dividends
if not dividends.empty:
    # 直近の配当利回りを取得
    current_dividend_yield = info.get('dividendYield')
    if current_dividend_yield is not None:
        print(f"現在の配当利回り: {current_dividend_yield:.2%}")

    # 1年前の配当利回りを計算するための株価と配当情報を取得
    # 過去1年間の終値の平均を概算として利用
    # 配当利回り = (1株あたりの年間配当金 / 株価) * 100
    if len(hist) >= 250: # 約1年分のデータがあるか確認 (営業日換算)
        # 過去1年間の配当合計を計算
        # yfinanceのdividendsは配当落ち日に支払われた配当額を示す
        one_year_ago = (pd.Timestamp.now() - pd.DateOffset(years=1)).normalize()
        recent_dividends = dividends[dividends.index >= one_year_ago]
        total_dividends_past_year = recent_dividends.sum()

        if total_dividends_past_year > 0:
            # 1年前の株価を概算 (1年前の日付に最も近いデータを使用)
            one_year_ago_price_data = hist[hist.index >= one_year_ago].iloc[0]
            one_year_ago_price = one_year_ago_price_data['Close']

            # 1年前の配当利回りを計算
            one_year_ago_dividend_yield = (total_dividends_past_year / one_year_ago_price)
            print(f"約1年前の配当利回り: {one_year_ago_dividend_yield:.2%}")

            # 変動率を計算
            if current_dividend_yield is not None:
                change_rate_dividend_yield = (current_dividend_yield - one_year_ago_dividend_yield) / one_year_ago_dividend_yield
                print(f"配当利回りの1年変動率: {change_rate_dividend_yield:.2%}")
            else:
                print("現在の配当利回りが取得できないため、変動率は計算できません。")
        else:
            print("過去1年間に配当支払いがありませんでした。")
    else:
        print("過去1年分の株価データが不足しているため、1年前の配当利回りを計算できません。")
else:
    print("配当履歴データがありません。")

# 4. 配当性向の1年単位での変動率
print("\n--- 4. 配当性向の1年単位での変動率 ---")
# infoの中に'payoutRatio'というキーで配当性向が含まれている
current_payout_ratio = info.get('payoutRatio')
if current_payout_ratio is not None:
    print(f"現在の配当性向: {current_payout_ratio:.2%}") # パーセント表示
else:
    print("現在の配当性向データがありません。")

# 配当性向の過去データはyfinanceのinfoからは直接取得できません。
# そのため、決算情報（income statement）からEPS（一株あたり利益）と配当総額を取得し、
# 手動で計算する必要がありますが、yfinanceのfree版では詳細な過去のIncome Statementデータが
# 取得できない場合があるため、ここでは現在の情報のみに留めます。
# もしより詳細な分析が必要な場合は、有料APIや別のデータソースの利用を検討してください。

print("注: yfinanceの無料データでは、過去の配当性向の推移を直接追うのは困難です。")
print("   過去の配当性向の変動率を正確に把握するには、別途決算情報を取得し、")
print("   EPS（一株あたり利益）とDPS（一株あたり配当）から計算する必要があります。")
print("   または、有料の金融データサービスをご検討ください。")

--- 1. 株価データ (過去5日分) ---
                             Open    High     Low   Close    Volume  Dividends  Stock Splits
Date                                                                                        
2025-06-10 00:00:00+09:00  2653.0  2663.5  2640.0  2657.5  19556800        0.0           0.0
2025-06-11 00:00:00+09:00  2670.0  2678.0  2652.0  2656.0  16367500        0.0           0.0
2025-06-12 00:00:00+09:00  2645.0  2648.0  2608.0  2616.5  22784400        0.0           0.0
2025-06-13 00:00:00+09:00  2586.0  2589.0  2539.0  2555.0  36054800        0.0           0.0
2025-06-16 00:00:00+09:00  2580.5  2586.0  2554.0  2554.5  19281900        0.0           0.0


--- 2. 企業情報・指標 ---
企業名: Toyota Motor Corporation
業種: Auto Manufacturers
現在の株価: 2554.5
PER (株価収益率): 7.11
PBR (株価純資産倍率): 0.93

--- 3. 配当利回りの1年単位での変動率 ---
現在の配当利回り: 372.00%
過去1年分の株価データが不足しているため、1年前の配当利回りを計算できません。

--- 4. 配当性向の1年単位での変動率 ---
現在の配当性向: 25.03%
注: yfinanceの無料データでは、過去の配当性向の推移を直接追うのは困難です。
   過去の配当性向の変動率を正確に把握するには、別

In [5]:
pip install --upgrade yfinance



[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import yfinance as yf
import pandas as pd
import datetime

# Pandasの表示オプションを設定
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# トヨタ自動車 (7203.T) のデータを取得
ticker_code = '7203.T'
ticker = yf.Ticker(ticker_code)

# 1. 株価データの取得 (過去1年分の日足)
print("--- 1. 株価データ (過去5日分) ---")
# 1年分のデータを取得して、配当利回り計算に利用
# 期間を明確に指定することで、データの取得漏れを防ぐ
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=365 * 2) # 念のため2年分のデータを取得してみる
hist = ticker.history(start=start_date, end=end_date)
print(hist.tail()) # .tail()で末尾5日分を表示

print("\n" + "="*50 + "\n")

# 2. 企業情報・指標の取得
print("--- 2. 企業情報・指標 ---")
info = ticker.info

# 必要な情報を抽出して表示
print(f"企業名: {info.get('longName')}")
print(f"業種: {info.get('industry')}")
print(f"現在の株価: {info.get('currentPrice')}")
print(f"PER (株価収益率): {info.get('trailingPE'):.2f}" if info.get('trailingPE') is not None else "PER: データなし")
print(f"PBR (株価純資産倍率): {info.get('priceToBook'):.2f}" if info.get('priceToBook') is not None else "PBR: データなし")

# 3. 配当利回りの1年単位での変動率
print("\n--- 3. 配当利回りの1年単位での変動率 ---")
# 過去の配当履歴を取得
dividends = ticker.dividends
if not dividends.empty:
    # 現在の配当利回りを取得 (念のためNoneチェックを追加)
    current_dividend_yield = info.get('dividendYield')
    if current_dividend_yield is not None:
        print(f"現在の配当利回り: {current_dividend_yield:.2%}")
    else:
        print("現在の配当利回りデータがありません。")

    # 1年前の配当利回りを計算するための株価と配当情報を取得
    # 過去1年間の終値の平均を概算として利用
    # 配当利回り = (1株あたりの年間配当金 / 株価) * 100

    # 約1年前の株価を取得する際の基準日
    one_year_ago_date = pd.Timestamp.now() - pd.DateOffset(years=1)

    # 過去1年間の配当合計を計算
    # yfinanceのdividendsは配当落ち日に支払われた配当額を示す
    recent_dividends = dividends[dividends.index >= one_year_ago_date]
    total_dividends_past_year = recent_dividends.sum()

    if total_dividends_past_year > 0:
        # 1年前の株価を概算 (1年前の日付に最も近いデータを使用)
        # histのインデックスから、one_year_ago_dateに最も近い日付のデータを探す
        if not hist.empty:
            # 1年前の日付以降の最初の株価データを取得
            one_year_ago_price_data_candidates = hist[hist.index >= one_year_ago_date]
            if not one_year_ago_price_data_candidates.empty:
                one_year_ago_price = one_year_ago_price_data_candidates.iloc[0]['Close']

                # 1年前の配当利回りを計算
                one_year_ago_dividend_yield = (total_dividends_past_year / one_year_ago_price)
                print(f"約1年前の配当利回り: {one_year_ago_dividend_yield:.2%}")

                # 変動率を計算
                if current_dividend_yield is not None and one_year_ago_dividend_yield > 0:
                    change_rate_dividend_yield = (current_dividend_yield - one_year_ago_dividend_yield) / one_year_ago_dividend_yield
                    print(f"配当利回りの1年変動率: {change_rate_dividend_yield:.2%}")
                elif current_dividend_yield is not None and one_year_ago_dividend_yield == 0:
                    print("1年前の配当利回りがゼロのため、変動率は計算できません。")
                else:
                    print("現在の配当利回りが取得できないため、変動率は計算できません。")
            else:
                print("過去1年間の株価データが見つからないため、1年前の配当利回りを計算できません。")
        else:
            print("株価データが取得できませんでした。")
    else:
        print("過去1年間に配当支払いがありませんでした。")
else:
    print("配当履歴データがありません。")

# 4. 配当性向の1年単位での変動率
print("\n--- 4. 配当性向の1年単位での変動率 ---")
# infoの中に'payoutRatio'というキーで配当性向が含まれている
current_payout_ratio = info.get('payoutRatio')
if current_payout_ratio is not None:
    print(f"現在の配当性向: {current_payout_ratio:.2%}") # パーセント表示
else:
    print("現在の配当性向データがありません。")

print("注: yfinanceの無料データでは、過去の配当性向の推移を直接追うのは困難です。")
print("   過去の配当性向の変動率を正確に把握するには、別途決算情報を取得し、")
print("   EPS（一株あたり利益）とDPS（一株あたり配当）から計算する必要があります。")
print("   または、有料の金融データサービスをご検討ください。")

--- 1. 株価データ (過去5日分) ---
                             Open    High     Low   Close    Volume  Dividends  Stock Splits
Date                                                                                        
2025-06-09 00:00:00+09:00  2670.0  2672.0  2637.0  2640.0  13584200        0.0           0.0
2025-06-10 00:00:00+09:00  2653.0  2663.5  2640.0  2657.5  19556800        0.0           0.0
2025-06-11 00:00:00+09:00  2670.0  2678.0  2652.0  2656.0  16367500        0.0           0.0
2025-06-12 00:00:00+09:00  2645.0  2648.0  2608.0  2616.5  22784400        0.0           0.0
2025-06-13 00:00:00+09:00  2586.0  2589.0  2539.0  2555.0  36054800        0.0           0.0


--- 2. 企業情報・指標 ---
企業名: Toyota Motor Corporation
業種: Auto Manufacturers
現在の株価: 2554.5
PER (株価収益率): 7.11
PBR (株価純資産倍率): 0.93

--- 3. 配当利回りの1年単位での変動率 ---
現在の配当利回り: 372.00%


TypeError: Invalid comparison between dtype=datetime64[ns, Asia/Tokyo] and Timestamp